# PYNQ Car using DFROBOT Turtle: 2WD Mobile Robot Platform

<img src="./images/pynq_car.jpg" style="width:400px;" />

---

## Aim

* This notebook illustrates how to use available APIs for the Geared Motor and DC Motor Driver on PYNQ-Z2 PMOD interface and build the PYNQ Car using DFROBOT Turtle: 2WD Mobile Robot Platform.

## Peripherals
| Module                                                       | 	Quantity |
| ------------------------------------------------------------ | ------ |
| [Turtle: 2WD Mobile Robot Platform for Arduino](https://www.dfrobot.com/product-65.html) | 1      |
| [Gravity: Smart Grayscale Sensor](https://www.dfrobot.com/product-1007.html) | 2      |
| [Gravity: URM09 Ultrasonic Sensor (Trig)](https://www.dfrobot.com/product-2172.html) | 1      |
| [7.4V Lipo 2500mAh Battery (Arduino Power Jack)](https://www.dfrobot.com/product-489.html) | 1      |
| [Gravity: 2x1.2A DC Motor Driver with Gravity Connector (TB6612FNG)](https://www.dfrobot.com/product-1705.html) | 1      |
| [Grove - 4 pin Male Jumper to Grove 4 pin Conversion Cable (5 PCs per Pack)](https://www.seeedstudio.com/Grove-4-pin-Male-Jumper-to-Grove-4-pin-Conversion-Cable-5-PCs-per-Pack.html)| 1    |
| [Breadboard Jumper Cables High Quality (30 Pack)](https://www.dfrobot.com/product-516.html)|1|
| [Gravity: TT Motor Encoders Kit](https://www.dfrobot.com/product-98.html) (Optional) | 1 |
| [9g Metal Gear Micro Servo (1.8Kg )](https://www.dfrobot.com/product-1338.html)(Optional) | 1 |

## Last revised
* 21 Oct 2021
    + DFRobot sensor version
---

## 1.Load _base_ Overlay

In [23]:
from pynq.overlays.base import BaseOverlay
from pynq_peripherals import ArduinoDIGILENTGroveAdapter, PmodGroveAdapter
base = BaseOverlay('base.bit')

<!-- ## Control the motor with PYNQ PMOD -->
## 2. Make Physical Connections

<div class="alert alert-box alert-warning"><ul>
    <h4 class="alert-heading">Make Physical Connections of Motors</h4>
    <li>Connect the motors to motor driver and the motor driver should be powered by a DC source larger than 6V. </li><li>Connect the green wires from motor driver PWM1 and PWM2 to PMODA pin 2 and PMODB pin 2 respectively, connect the green wire of DIR1 and DIR2 to PMODA pin 6 and PMODB pin 6 repectively. </li><li> The black wires from motor driver should connect to PMOD ground pins.</li><li> The red wires from motor driver are left floating, hence they can be unplugged or cut from the connector.</li>
</ul>
</div>

<img src="./images/motor_system.png"  style="width:450px;" />

<div class="alert alert-box alert-warning"><ul>
    <h4 class="alert-heading">Make Physical Connections of Sensors</h4>
    <li>Insert the Digilent Grove Base Shield into the Arduino connector on the board. Connect two Smart Grayscale Sensors to G5 and G6 connector of the Digilent Grove Base Shield respectively.</li>
    <li>Connect the Usranger module to G4 connector of the Grove Base Shield</li>
    <li>The two line finders should be aligned together in front of the turtle.</li>
</ul>
</div>


<img src="images/sensor_connection.png" style="width:450px;" />

<div class="alert alert-box alert-info"><ul>
    <h4 class="alert-heading">Notes: Turn off the motor power now to avoid unexpected motion during configuration</h4>
</ul>
</div>

# 3. Configure adapter and peripherals

### Adapter configuration

In [8]:
adapter_A = PmodGroveAdapter(base.PMODA, G4='geared_motor')
adapter_B = PmodGroveAdapter(base.PMODB, G4='geared_motor')
arduino_adapter = ArduinoDIGILENTGroveAdapter(base.ARDUINO, G6='grove_line_finder',G5='grove_line_finder',G4='grove_usranger')

### Define motor object

<div class="alert alert-box alert-info"><ul>
    <h4 class="alert-heading">Notes: </h4>
    <li><b>Set the direction:</b>  Use  forward() and  backforward() function to go forward or backward respectively.</li>
    <li><b>Set the speed:</b> Valid values are from -100 to 100. The car will go forward if the speed is positive and go backword if the speed is negative.  </li>
</ul>
</div>

In [10]:
right_motor = adapter_A.G4
left_motor = adapter_B.G4
right_motor.backward()
left_motor.backward()
right_motor.set_speed(0)
left_motor.set_speed(0)

### Define sensor object

In [12]:
left_line_finder = arduino_adapter.G5
right_line_finder = arduino_adapter.G6
usranger = arduino_adapter.G4

# 4. Test peripherals

### Calibrate smart grayscale sensors
Just short click the button and the mode will be changed between analog and digital.

Here, we use the digital mode (detailed instruction could be found in the [specification](https://wiki.dfrobot.com/Smart_Grayscale_sensor_SKU_SEN0147) of smart grayscale sensor).

<div class="alert alert-box alert-warning"><ul>
    <h4 class="alert-heading">Set the logic of two different color by your own</h4>
    <li> Long click the button until the LED slow blinking (which means first color detecting ready). Let the grayscale sensor face to the first color (such as white paper). Meanwhile short click the button and the LED gets dark (which means first color detected).</li>
    <li> Short click the button and make the LED fast blink (which means second color detecting ready). Let the grayscale sensor face to the second color (such as black paper). Meanwhile short click the button and the LED gets dark (which means second color detected).</li>
    <li>The first color (such as white paper) will be Logical '1', and the LED becomes bright. The second color (such as black paper) will be Logical '0', and the LED becomes dark. This setting maintains even if the power is off, so for one purpose, one setting is enough.</li>
    </ul>
</div>

<img src="images/BlackLight.png" style="width:350px;" />

### Test sensors

- Put your hand in front of the Usranger to test the get_distance() function
- Smart Grayscale Sensor shall output various logics for black and white surface

In [18]:
print('distance: {:.2f} cm'.format(usranger.get_distance()))
print(left_line_finder.line_found())
print(right_line_finder.line_found())

distance: 500.00 cm
True
True


### Test motors

- The car should rotate for 3 seconds

<div class="alert alert-box alert-info"><ul>
    <h4 class="alert-heading">Turn on the motor power now.</h4>
</ul>
</div>

In [22]:
def rotate(speed):
    right_motor.forward()
    left_motor.backward()
    right_motor.set_speed(speed)
    left_motor.set_speed(speed)
    
def stop():
    left_motor.set_speed(0)
    right_motor.set_speed(0)
    left_motor.backward()
    right_motor.backward()
    
import time

rotate(40)
time.sleep(3)
stop()

## 5. PYNQ Car Competition

The following figure shows a sample map. Your car should be programmed to run from the **start line** to the **end line**.

Notes:
- Two smart grayscalse sensors shall be used to track black lines and adjust direction.
- One usranger shall be used to detect obstacle and help steering.

<div class="alert alert-box alert-info"><ul>
    <h4 class="alert-heading">Make a map and have a try!</h4>
</ul>
</div>

<img src="images/map.png" style="width:450px;" />

Copyright (C) 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----

----